In [1]:
pip install opencv-python numpy pyautogui

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (

In [ ]:
import cv2
import numpy as np
import pyautogui
from collections import deque
import time

class HandGestureRecognizer:
    def __init__(self):
        # Initialize webcam with optimal settings
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        self.cap.set(cv2.CAP_PROP_FPS, 30)
        
        # Define the region of interest
        self.roi_top = 100
        self.roi_bottom = 300
        self.roi_right = 300
        self.roi_left = 100
        
        # Skin color range in YCrCb color space
        self.skin_ycrcb_min = np.array((0, 135, 85))
        self.skin_ycrcb_max = np.array((255, 180, 135))
        
        # Gesture smoothing
        self.gesture_history = deque(maxlen=5)
        self.last_gesture_time = time.time()
        self.gesture_cooldown = 0.3
        
        # Command mapping
        self.gesture_commands = {
            'FIVE': {'command': 'space', 'min_confidence': 0.8},
            'FIST': {'command': 'esc', 'min_confidence': 0.85},
            'TWO': {'command': 'right', 'min_confidence': 0.75}
        }

    def detect_skin(self, frame):
        # Convert to YCrCb color space
        ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCR_CB)
        
        # Create skin mask
        skin_mask = cv2.inRange(ycrcb, self.skin_ycrcb_min, self.skin_ycrcb_max)
        
        # Apply morphological operations
        kernel = np.ones((3, 3), np.uint8)
        skin_mask = cv2.erode(skin_mask, kernel, iterations=2)
        skin_mask = cv2.dilate(skin_mask, kernel, iterations=2)
        skin_mask = cv2.GaussianBlur(skin_mask, (5, 5), 0)
        
        return skin_mask

    def analyze_contour(self, contour, frame_shape):
        if len(contour) < 5:
            return None, 0
            
        # Calculate basic contour properties
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        hull = cv2.convexHull(contour)
        hull_area = cv2.contourArea(hull)
        
        # Calculate shape features
        solidity = float(area) / hull_area if hull_area > 0 else 0
        
        # Get convexity defects
        hull_indices = cv2.convexHull(contour, returnPoints=False)
        try:
            defects = cv2.convexityDefects(contour, hull_indices)
        except:
            return None, 0
            
        if defects is None:
            return None, 0
            
        # Count fingers using defect analysis
        finger_count = self.count_fingers(defects, contour, frame_shape)
        
        # Calculate confidence based on multiple factors
        confidence = self.calculate_confidence(solidity, area, finger_count, frame_shape)
        
        return finger_count, confidence

    def count_fingers(self, defects, contour, frame_shape):
        finger_count = 1
        frame_area = frame_shape[0] * frame_shape[1]
        
        # Get contour center
        moments = cv2.moments(contour)
        if moments['m00'] == 0:
            return 0
            
        cx = int(moments['m10'] / moments['m00'])
        cy = int(moments['m01'] / moments['m00'])
        
        valid_defects = 0
        for i in range(defects.shape[0]):
            s, e, f, d = defects[i, 0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])
            
            # Calculate angles and distances
            a = np.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = np.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = np.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            
            angle = np.degrees(np.arccos((b**2 + c**2 - a**2)/(2*b*c)))
            
            # Distance from defect to center
            far_distance = np.sqrt((far[0] - cx)**2 + (far[1] - cy)**2)
            
            # Filter valid finger defects
            if angle <= 90 and far_distance > 30 and d/256 > frame_area/20000:
                valid_defects += 1
                
        finger_count += valid_defects
        return min(finger_count, 5)

    def calculate_confidence(self, solidity, area, finger_count, frame_shape):
        # Calculate frame area for relative measurements
        frame_area = frame_shape[0] * frame_shape[1]
        
        # Area-based confidence
        area_confidence = min(area / (frame_area/4), 1.0)
        
        # Shape-based confidence
        shape_confidence = solidity
        
        # Temporal confidence
        temporal_confidence = self.calculate_temporal_confidence(finger_count)
        
        # Combined confidence score
        confidence = (0.4 * area_confidence + 
                     0.3 * shape_confidence +
                     0.3 * temporal_confidence)
        
        return confidence

    def calculate_temporal_confidence(self, current_gesture):
        if len(self.gesture_history) < self.gesture_history.maxlen:
            return 0.5
            
        matching_gestures = sum(1 for g in self.gesture_history 
                              if g == current_gesture)
        return matching_gestures / len(self.gesture_history)

    def process_frame(self, frame):
        # Extract region of interest
        roi = frame[self.roi_top:self.roi_bottom, 
                   self.roi_left:self.roi_right]
        
        # Detect skin in ROI
        skin_mask = self.detect_skin(roi)
        
        # Find contours
        contours, _ = cv2.findContours(skin_mask, 
                                     cv2.RETR_EXTERNAL, 
                                     cv2.CHAIN_APPROX_SIMPLE)
        
        # Process if contours are found
        if contours:
            # Get largest contour
            max_contour = max(contours, key=cv2.contourArea)
            
            # Analyze contour
            finger_count, confidence = self.analyze_contour(
                max_contour, roi.shape)
            
            if finger_count is not None:
                # Update gesture history
                self.gesture_history.append(finger_count)
                
                # Execute command if confidence is high
                current_time = time.time()
                if (current_time - self.last_gesture_time >= 
                    self.gesture_cooldown):
                    gesture_map = {
                        5: 'FIVE',
                        0: 'FIST',
                        2: 'TWO'
                    }
                    
                    if finger_count in gesture_map:
                        gesture = gesture_map[finger_count]
                        if (gesture in self.gesture_commands and 
                            confidence >= 
                            self.gesture_commands[gesture]['min_confidence']):
                            pyautogui.press(
                                self.gesture_commands[gesture]['command'])
                            self.last_gesture_time = current_time
                
                # Draw visualization
                cv2.drawContours(roi, [max_contour], -1, (0, 255, 0), 2)
                hull = cv2.convexHull(max_contour)
                cv2.drawContours(roi, [hull], -1, (0, 0, 255), 2)
                
                # Display information
                cv2.putText(frame, 
                           f'Fingers: {finger_count} (Conf: {confidence:.2f})',
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                           1, (255, 0, 0), 2)
        
        # Draw ROI rectangle
        cv2.rectangle(frame, 
                     (self.roi_left, self.roi_top),
                     (self.roi_right, self.roi_bottom),
                     (255, 0, 0), 2)
        
        return frame

    def run(self):
        try:
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    print("Failed to grab frame")
                    break
                    
                # Flip frame horizontally
                frame = cv2.flip(frame, 1)
                
                # Process frame
                processed_frame = self.process_frame(frame)
                
                # Display result
                cv2.imshow('Hand Gesture Recognition', processed_frame)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        except Exception as e:
            print(f"Error during execution: {str(e)}")
        finally:
            self.cleanup()

    def cleanup(self):
        self.cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    recognizer = HandGestureRecognizer()
    recognizer.run()